## Data cleaning & EDA

In [153]:
import re
import pandas as pd

In [ ]:
df_benign = pd.read_csv('../Data/CSV_benign.csv')
df_malware = pd.read_csv('../Data/CSV_malware.csv')

# 'Country' column name is duplicated in malware csv, therefore I decided to rename both. While reading it, pandas reads duplicated column name with '.1' suffix
df_benign.rename(columns={'Country.1':'Country_1'}, inplace=True)
df_malware.rename(columns={'Country.1':'Country_1'}, inplace=True)

# Reindex columns
df_benign = df_benign.reindex(sorted(df_benign.columns), axis=1)
df_malware = df_malware.reindex(sorted(df_malware.columns), axis=1)
# df_malware.head(3)

In [ ]:
# By digging into missmatch in dtypes, I was able to identify 'mixing' of columns data in 24 records of df_malware.
# To fix it, following steps are taken: 1. Identify incorrect rows by checking len of IP column,
#   2. Get records into new df
#   3. Rename columns
#   4. Drop incorrect rows from df_malware
#   5. Concatenate fixed data to df_malware

incorrect_rows_idx = df_malware.index[df_malware['IP'].str.len()==2]
df_incorrect_rows = df_malware.iloc[incorrect_rows_idx]

# Applies to df_malware only - rename of columns for 24 records
col_val_replace_to = {
    'Country': 'TTL',
    'TTL': 'Domain',
    'IP': 'Country',
    'Domain': 'IP',
}

df_incorrect_rows.rename(columns=col_val_replace_to, inplace=True) # Apply rename
df_malware.drop(incorrect_rows_idx, axis=0, inplace=True) # Drop from malware df incorrect rows
df_malware = pd.concat([df_malware, df_incorrect_rows], ignore_index=False) # Concatenate fixed data

In [156]:
# To mitigate missing values across similar columns like Domain, Domain_Name and Country, Country_1, following code is applied to df's
# The code also applies mapping to unify a bit entries
countries_map = {
    '-':'',
    "china":"CN",
    "Malaysia":'ID',
    "United States":"US",
    "TURKEY":'TR',
    'RUSSIA':'RU',
    'Russian Federation':'RU',
    'Belarus':'BY',
    'Korea':'KR',
}

def use_regex(input_text):
    return re.sub(r"b'(.+?).'", r"\1", input_text)

def impute_similar_cols(df):
    df["Country_1"].replace(countries_map, inplace=True)
    df["Country"].replace(countries_map, inplace=True)
    df["Country_1"].fillna(df["Country"], inplace=True)
    df["Country"].fillna(df["Country_1"], inplace=True)
    df["Domain_Name"].fillna(df["Domain"].apply(use_regex), inplace=True)
    df['Domain_Age'] = df['Domain_Age'].str.split(' ').str[0]
    return df

df_malware = impute_similar_cols(df_malware)
df_benign = impute_similar_cols(df_benign)

In [157]:
print(df_malware.shape, df_benign.shape)

(4999, 38) (494135, 38)


In [158]:
df_benign['is_threat'] = 0
df_malware['is_threat'] = 1

# Combine two dataframes
df = pd.concat([df_benign, df_malware])
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# Manually select informative fields based on # of nulls it contain
percent_na = (df.isna().sum() / len(df)) * 100
columns_to_keep = percent_na[percent_na <= 30].index.tolist()
df = df[columns_to_keep]
df.dropna(axis=0, inplace=True)
df.drop_duplicates(inplace=True)

print('df shape: ', df.shape)
print(df.columns)

df shape:  (244072, 35)
Index(['1gram', '2gram', '3gram', 'ASN', 'Alexa_Rank', 'Country', 'Country_1',
       'Creation_Date_Time', 'Domain', 'Domain_Age', 'Domain_Name', 'IP',
       'Name_Server_Count', 'Page_Rank', 'Registrar', 'TTL',
       'char_distribution', 'dec_32', 'dec_8', 'entropy', 'hex_32', 'hex_8',
       'len', 'longest_word', 'numeric_percentage', 'obfuscate_at_sign',
       'oc_32', 'oc_8', 'puny_coded', 'shortened', 'sld', 'subdomain', 'tld',
       'typos', 'is_threat'],
      dtype='object')


In [159]:
# df.to_csv('./tmp/test.csv')

In [160]:
# Manual features selection based on previous experiments, numeric ones only
selected_cols = ['Alexa_Rank','ASN','Domain_Age','TTL','entropy','len','numeric_percentage','subdomain']
selected_cols += ['is_threat']

df = df[selected_cols]

def cols_to_num_drop(df, cols):
    for col in cols:
        df.loc[:,col] = pd.to_numeric(df[col], errors='coerce') # Should be number
        df.dropna(subset=[col], inplace=True)

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


def cols_to_num_fill(df, cols):
    for col in cols:
        df.loc[:,col] = pd.to_numeric(df[col], errors='coerce') # Should be number
        df.loc[:,col].fillna(0, inplace = True)
    return df

df = cols_to_num_drop(df,selected_cols)
df.shape

(215120, 9)

In [161]:
print(f"{df['is_threat'].value_counts()[1]/df['is_threat'].value_counts()[0]*100:.3f}% of whole dataset domains are a threat")

1.285% of whole dataset domains are a threat


In [178]:
def train_valid_singleframe_proportional(df, ratio = 0.1):
    train = df.iloc[:int(len(df) * (1 - ratio))]
    valid = df.iloc[int(len(df) * (1 - ratio)):]
    return (train, valid)

def train_valid_singleframe_n_last(df, n_samples = 100):
    train = df.iloc[:int(len(df) - n_samples)]
    valid = df.iloc[int(len(df) - n_samples):]
    return (train, valid)

def get_validset(df, type):
    unique_groups = df['is_threat'].unique()
    ret_train = pd.DataFrame()
    ret_valid = pd.DataFrame()

    for group in unique_groups:
        selected_df = df[df['is_threat']==group]

        if type=='n_last':
            train, valid = train_valid_singleframe_n_last(selected_df)
        elif type=='proportional':
            train, valid = train_valid_singleframe_proportional(selected_df)
        else:
            raise('Only "n_last" or "proportional" is available')

        ret_train = pd.concat([ret_train, train], ignore_index=False)
        ret_valid = pd.concat([ret_valid, valid], ignore_index=False)

    return (ret_train, ret_valid)


df, df_valid_n_last = get_validset(df, 'n_last')

## References

Samaneh Mahdavifar, Nasim Maleki, Arash Habibi Lashkari, Matt Broda, Amir H. Razavi, “Classifying Malicious Domains using DNS Traffic Analysis”, The 19th IEEE International Conference on Dependable, Autonomic, and Secure Computing (DASC), Oct. 25-28, 2021, Calgary, Canada